### LangGraph Agent - Customer Support multivoice Agent

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are Andrea, a knowledgeable and friendly assistant in a telecommunications company. Your expertise lies in various mobile plans and upgrades. Your role is to help users understand their options and assist them with their queries about mobile plans and upgrades. Always respond in a helpful and professional manner.
Always speak to the user with his name.

Username: {username}
UserID: {user_id}

Remember, your goal is to make the user feel supported and informed. Always be courteous and clear in your responses.
"""



In [3]:
import requests
from langchain_core.tools import tool

API_URL = "http://127.0.0.1:8000"
ADMIN_USERNAME = "admin1"
ADMIN_PASSWORD = "admin1password"

@tool
def create_contract_tool(user_id: int, category: str):
    """
    Create a new contract for a user with a specific category.

    Args:
        user_id (int): ID of the user for whom the contract is being created.
        category (str): Category of the contract. Must be one of "basic", "normal", or "premium".

    Returns:
        str: A string containing the details of the created contract.
        str: Error message if the user is not found or if any other validation fails.

    This function interacts with the FastAPI API to create a new contract for the specified user.
    """
    try:
        # Login as admin
        login_response = requests.post(f"{API_URL}/token/", data={"username": ADMIN_USERNAME, "password": ADMIN_PASSWORD})
        if login_response.status_code != 200:
            return f"Login failed: {login_response.json().get('detail')}"

        access_token = login_response.json().get("access_token")
        headers = {"Authorization": f"Bearer {access_token}"}

        # Validate the category
        if category not in ["basic", "normal", "premium"]:
            return "Category must be one of: basic, normal, premium"

        # Create contract
        contract_data = {"user_id": user_id, "category": category}
        create_response = requests.post(f"{API_URL}/contracts/", json=contract_data, headers=headers)
        if create_response.status_code != 200:
            return f"Failed to create contract: {create_response.json().get('detail')}"

        contract = create_response.json()
        return f"Contract created: User ID {contract['user_id']}, Category {contract['category']}, Contract Time {contract['contract_time']}"
    except Exception as e:
        return f"Failed to execute. Error: {repr(e)}"

tools = [create_contract_tool]

In [4]:
from langchain_core.messages.human import HumanMessage

order_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("placeholder", "{chat_history}"),
        ("human", "{question}"),
    ]
)

question = "I would like to create a premium contract please. My user ID is 6"
chat_history = []
messages = [HumanMessage(content=question)]

model = ChatOpenAI()
model_with_tools = model.bind_tools(tools=tools)
order_checker_llm = order_prompt | model_with_tools
result = order_checker_llm.invoke(
    {"question": question, "chat_history": chat_history, "username": "Max", "user_id": 3},
)

In [5]:
messages.append(result)

In [6]:
result.tool_calls

[{'name': 'create_contract_tool',
  'args': {'user_id': 6, 'category': 'premium'},
  'id': 'call_8Si7qdj9JzdUVeHy0Mvud2TA',
  'type': 'tool_call'}]

In [7]:
from langchain_core.messages import ToolMessage

for tool_call in result.tool_calls:
    print("Use Tool:", tool_call)
    selected_tool = tool_mapping = {"create_contract_tool": create_contract_tool}[
        tool_call["name"].lower()
    ]
    tool_output = selected_tool.invoke(tool_call["args"])
    print(tool_output)
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

Use Tool: {'name': 'create_contract_tool', 'args': {'user_id': 6, 'category': 'premium'}, 'id': 'call_8Si7qdj9JzdUVeHy0Mvud2TA', 'type': 'tool_call'}
Failed to create contract: User already has a contract


In [8]:
messages

[HumanMessage(content='I would like to create a premium contract please. My user ID is 6'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8Si7qdj9JzdUVeHy0Mvud2TA', 'function': {'arguments': '{"user_id":6,"category":"premium"}', 'name': 'create_contract_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 268, 'total_tokens': 288}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1be3cc07-5351-4346-b652-d8ae17afa307-0', tool_calls=[{'name': 'create_contract_tool', 'args': {'user_id': 6, 'category': 'premium'}, 'id': 'call_8Si7qdj9JzdUVeHy0Mvud2TA', 'type': 'tool_call'}], usage_metadata={'input_tokens': 268, 'output_tokens': 20, 'total_tokens': 288}),
 ToolMessage(content='Failed to create contract: User already has a contract', tool_call_id='call_8Si7qdj9JzdUVeHy0Mvud2TA')]

In [10]:
model_with_tools.invoke(messages)

AIMessage(content='It seems that you already have a contract. If you have any specific requests or need assistance with your existing contract, please let me know how I can help you further.', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 212, 'total_tokens': 247}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-01ee8b49-b773-47d2-b14f-365d8e971116-0', usage_metadata={'input_tokens': 212, 'output_tokens': 35, 'total_tokens': 247})

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]
    ask_human: bool
    agent_name: str

In [ ]:
from langchain_core.tools import tool


@tool
def request_assistance():
    """Escalate the conversation to an expert. Use this if you are unable to assist directly or if the user requires support beyond your permissions.

    To use this function, relay the user's 'request' so the expert can provide the right guidance.
    """
    return ""